<a href="https://colab.research.google.com/github/aayushkumar20/ML-based-mask-checker./blob/main/code/mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import cv2
import time
import os

def detect_and_predict_mask(frame,faceNet,maskNet):
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(224,224),(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections=faceNet.foreward(blob)
    print(detections.shape)
    faces=[]
    locs=[]
    preds=[]
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX),(w-1,endY))
            face=frame[startY:endY,startX:endX]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RBG)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)

            faces.append(face)
            locs.append((startX,startY,endX,endY))

    if len(faces)>0:
        faces=np.array(faces,dtype="float32")
        faces=preprocess_input(faces)
        preds=maskNet.predict(faces,batch_size=32)
    return(locs,preds)

prototxtPath="face_detector/deploy.prototxt"
weightsPath="face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)
maskNet=load_model("mask_detector/mask_detector.model")

print("[INFO] starting")
vs=VideoStream(src=0).start()

while True:
    frame=vs.read()
    frame=imutils.resize(frame,width=400)
    (locs,preds)=detect_and_predict_mask(frame,faceNet,maskNet)
    for (box,preds) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=preds
        label="mask" if mask>withoutMask else "There's no mask"
        color=(0,255,0) if label=="mask" else(0,255,0)
        label="{}:{:.2f}%".format(label,max(mask,withoutMask)*1000)
        cv2.putText(frame,label,(startX,startY-18),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    cv2.imshow("frame",frame)
    key=cv2.waitKey(1) & 0xff
    if key==ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()